<a href="https://colab.research.google.com/github/compartia/AI-tecture/blob/master/miro_openai_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare OPEN AI client

In [ ]:
!pip install --upgrade openai typing-extensions

In [ ]:
import openai
from google.colab import userdata
# openai.api_key =

ai_client = openai.OpenAI(api_key=userdata.get('OP_API')  )

In [33]:
BOARD_ID = 'uXjVN61x8Pg='
DEBUG_OPENAI = False

# Test open AI

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
#

### Test gpt api

In [ ]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )

  # print(completion.choices[0].message)
  display(HTML(completion.choices[0].message.content))

In [ ]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )
  #
  # print(completion.choices[0].message)
  display(HTML(completion.choices[0].message.content))

In [ ]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )

  display(HTML(completion.choices[0].message.content))

## open AI api client code

In [ ]:
def ask_ai(prompt, argument_context):

  completion = ai_client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": f"{prompt}. Your answer is in a form of list of <p> html tags and NO newline '\n' symbols."},
      {"role": "user", "content": argument_context}
    ]
  )
  return completion.choices[0].message.content

if DEBUG_OPENAI:
  r = ask_ai('you are the philosopher. answer in 50 words.', 'what is the meaning of life?')
  print(r)
  display(HTML(r))
  # display(HTML(r))

# Miro client

In [146]:
import requests
import pandas as pd


class MiroClient:
  def __init__(self, token, board_id):
    self.board_id = board_id
    self.token = token
    self.base_url = f"https://api.miro.com/v2/boards/{board_id}"

    self.headers = {
        "accept": "application/json",
        "authorization": f"Bearer {self.token}"
    }


  def get_items(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/items?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/items?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response

  def get_connectors(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/connectors?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/connectors?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response


  def get_all_pages(self, page_method):
    # print('get_all_pages, page_method=', page_method)
    cursor = None

    while True:
      response = page_method(cursor)
      rj = response.json()
      yield rj
      cursor = rj.get('cursor')
      if cursor is None:
        #TODO: safety! add limit : we trust 3rd pary apy too much, what if cursor is never None?
        break


miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)


In [147]:
r = miro_client.get_items(limit=50).json()

for k in r['data']:
  print()
  print('-'*30)
  print(k)


------------------------------
{'id': '3458764575387244511', 'type': 'sticky_note', 'data': {'content': '<p>#history</p>', 'shape': 'rectangle'}, 'style': {'fillColor': 'light_yellow', 'textAlign': 'center', 'textAlignVertical': 'middle'}, 'geometry': {'width': 156.0758892005528, 'height': 101.67229353636012}, 'parent': {'links': {'self': 'http://api.miro.com/v2/boards/uXjVN61x8Pg%3D/items/3458764575504154876'}, 'id': '3458764575504154876'}, 'position': {'x': 1124.2968541484329, 'y': 1126.3503896355378, 'origin': 'center', 'relativeTo': 'parent_top_left'}, 'links': {'self': 'http://api.miro.com/v2/boards/uXjVN61x8Pg%3D/sticky_notes/3458764575387244511'}, 'createdAt': '2024-01-12T08:04:13Z', 'createdBy': {'id': '3458764575386404073', 'type': 'user'}, 'modifiedAt': '2024-01-28T09:55:00Z', 'modifiedBy': {'id': '3458764575386404073', 'type': 'user'}}

------------------------------
{'id': '3458764575388345911', 'type': 'shape', 'data': {'content': '<p><strong>#innertia</strong></p>', 'sha

In [148]:
type(r['data'])

list

## Getting all connectors

In [149]:
def decode_connectors(page:dict):
  # print(page)
  for i in page['data']:
    yield i['id'], i.get('startItem', {}).get('id'), i.get('endItem', {}).get('id')

## test it
one_page_con = miro_client.get_connectors(limit=10).json()
x = [i for i in decode_connectors(one_page_con)]
x

[('3458764575387926040', '3458764575403520160', '3458764575403520159'),
 ('3458764575388345046', '3458764575403702754', '3458764575441998372'),
 ('3458764575402117332', '3458764575387244511', '3458764575403520158'),
 ('3458764575402617877', '3458764575388345911', '3458764577059064018'),
 ('3458764575402701153', '3458764575388345911', '3458764575403520158'),
 ('3458764575402761714', '3458764575388345911', '3458764575403289322'),
 ('3458764575402761923', '3458764575387244511', '3458764575403289322'),
 ('3458764575402951927', '3458764575487075968', '3458764575487769026'),
 ('3458764575403289001', '3458764575487075968', '3458764575403289457'),
 ('3458764575405170736', '3458764575405170663', '3458764575404992843')]

In [150]:
def get_all_connectors(self):
  for page in self.get_all_pages(self.get_connectors):
    # print('get_all_connectors, page:', page)
    for i in decode_connectors(page):
      yield i


MiroClient.get_all_connectors = get_all_connectors


##-----
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
all_connectors_iter = miro_client.get_all_connectors()

connectors_df = pd.DataFrame(all_connectors_iter, columns=['id','id_from', 'id_to'])
connectors_df = connectors_df.set_index('id')
connectors_df


,id_from,id_to
id,,
3458764575387926040,3458764575403520160,3458764575403520159
3458764575388345046,3458764575403702754,3458764575441998372
3458764575402117332,3458764575387244511,3458764575403520158
3458764575402617877,3458764575388345911,3458764577059064018
3458764575402701153,3458764575388345911,3458764575403520158
...,...,...
3458764578213590397,3458764578194135331,3458764578250671685
3458764578216605248,3458764578079690837,3458764578250671917
3458764578223088948,3458764578250671685,3458764578223716357


## Getting all shapes (widgets)

In [151]:
def decode_items(page:dict):

  for i in page['data']:
    # print(type(i))
    if type(i) == dict :
      yield i['id'], i['type'], i['data'].get('shape', '_undefined_'), i['data'].get('content'), i['modifiedAt'], i['geometry'], i['position']

    else:
      print('NOT A DICT!!', type(i), i)
      decode_items(i)



def get_all_items(self):
  for page in self.get_all_pages(self.get_items):
    # print( page.get('cursor'), page.get('size'), page.get('total') )
    for i in decode_items(page):
      yield i
      # print(i)


MiroClient.get_all_items = get_all_items

In [152]:
### --------

# items_iter = miro_client.get_all_items()

# df = pd.DataFrame(items_iter, columns=['id','type', 'shape', "contents", 'modifiedAt', 'geometry', 'position'])
# df = df.set_index('id')
# shapes_df = df
# shapes_df

def all_items_as_df(miro_client):
  items_iter = miro_client.get_all_items()

  df = pd.DataFrame(items_iter, columns=['id','type', 'shape', "contents", 'modifiedAt', 'geometry', 'position'])
  df = df.set_index('id')
  shapes_df = df
  miro_client.shapes_df = shapes_df
  return miro_client.shapes_df


MiroClient.all_items_as_df = all_items_as_df
# --------

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
shapes_df = miro_client.all_items_as_df()

In [153]:
shapes_df

,type,shape,contents,modifiedAt,geometry,position
id,,,,,,
3458764575387244511,sticky_note,rectangle,<p>#history</p>,2024-01-28T09:55:00Z,"{'width': 156.0758892005528, 'height': 101.672...","{'x': 1124.2968541484329, 'y': 1126.3503896355..."
3458764575388345911,shape,rectangle,<p><strong>#innertia</strong></p>,2024-01-28T09:55:00Z,"{'width': 135.10626555260657, 'height': 51.329...","{'x': 1144.8602145010623, 'y': 798.34928043655..."
3458764575403289322,shape,rectangle,<p><strong>Inertia: Patterns through Generatio...,2024-01-28T10:32:08Z,"{'width': 442.36679516447094, 'height': 150.21...","{'x': 707.2729633197238, 'y': 316.906189505656..."
3458764575403289457,shape,rectangle,<p><strong>Vague Terms; Words Ambiguity</stron...,2024-01-28T10:32:08Z,"{'width': 442.3667951644703, 'height': 284.990...","{'x': 707.2729633197235, 'y': 752.803705698517..."
3458764575403520157,shape,rectangle,"<p><strong>3. Наследственность, Genetic text</...",2024-01-28T09:55:00Z,"{'width': 349.7213126392021, 'height': 287.977...","{'x': 762.0926390377875, 'y': 1285.14457411205..."
...,...,...,...,...,...,...
3458764578250456192,text,_undefined_,<p>prompt for Chat GPT</p>,2024-02-07T15:00:44Z,{'width': 137.0},"{'x': 12924.274611533334, 'y': -2909.995078272..."
3458764578250456453,text,_undefined_,<p>Answer of Chat GPT</p>,2024-02-07T15:07:53Z,{'width': 130.0},"{'x': 13158.858632836302, 'y': -3454.755614901..."
3458764578250456731,text,_undefined_,<p>Takes concatenated texts</p><p>as inputs</p>,2024-02-07T15:01:54Z,{'width': 166.0},"{'x': 13805.085973323692, 'y': -3232.289605789..."


### Update board item

In [154]:
def get_shape_info(self, shape_id):
  url = f"{self.base_url}/shapes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)

def get_stiky_note_info(self, shape_id):
  url = f"{self.base_url}/sticky_notes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)


def update_item(self, shape_id, item_type, content, shape=None, color = None):

  url = None

  payload = {
      "data": {
          "content": content
      }
  }


  if color is not None:
    payload['style']={}
    payload['style']['fillColor'] = color

    if item_type!='sticky_note':
      payload['style']['fillOpacity'] = "1.0"


  if item_type=='sticky_note':
    url = f"{self.base_url}/sticky_notes/{shape_id}"
  elif item_type=='shape':
    url = f"{self.base_url}/shapes/{shape_id}"
    if shape is not None:
      payload['data']['shape'] = shape

  print(payload)


  response = requests.patch(url, json=payload, headers=self.headers)
  print(response.text)
  return response



MiroClient.update_item = update_item
MiroClient.get_shape_info = get_shape_info
MiroClient.get_stiky_note_info = get_stiky_note_info

#-----

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

# if False:

  # miro_client.get_stiky_note_info('3458764578213963472')
# test_id = '3458764578216605194'
# miro_client.update_item( test_id, item_type=shapes_df.at[test_id, 'type'], content=completion.choices[0].message.content,  color='#ff0000')

In [155]:
def create_connector(self, form, to, text):
  import requests

  url = f"{self.base_url}/connectors"

  payload = {
      "startItem": {
          "id": form,
          "snapTo": "right"
      },
      "endItem": {
          "id": to,
          "snapTo": "auto"
      },
      "style": { "endStrokeCap": "arrow" },
      "shape": "curved",
      "captions": [{ "content": text }]
  }

  response = requests.post(url, json=payload, headers=self.headers)

  print(response.text)


def create_stiky_note(self, contents, pos):

  url = f"{self.base_url}/sticky_notes"

  payload = {
    "data": {
        "content": contents,
        "shape": "rectangle"
    },
    "style": {
        "fillColor": "pink",
        "textAlign": "left"
    },
    "position": {
        "x": pos[0],
        "y": pos[1]
    },
    "geometry": {
        "height": pos[2]
        # "width": pos[3]
    }
  }


  response = requests.post(url, json=payload, headers=self.headers)
  return response

MiroClient.create_stiky_note = create_stiky_note
MiroClient.create_connector = create_connector
# _____
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
# response = miro_client.create_stiky_note( "TESTCREATE", (20,20,400,400) )
# response.json()['id']

# Processing the board

**TODO:**
1. sort subgraph, to proritize, what propmps to process first
2. create output shape if propmpt node has no output connectors

In [156]:
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

### Get shapes with prompts

In [157]:
prompts_df = shapes_df[shapes_df['shape']=='parallelogram']
prompts_df

,type,shape,contents,modifiedAt,geometry,position
id,,,,,,
3458764578079690837,shape,parallelogram,"<p>criticise this, what is not correct or not ...",2024-02-07T16:03:17Z,"{'width': 263.84845322298946, 'height': 122.98...","{'x': 12878.87833563337, 'y': -2824.5761499224..."
3458764578193870607,shape,parallelogram,<p>Is it true or false? What do you think abou...,2024-02-07T08:11:55Z,"{'width': 231.0645545179327, 'height': 96.0998...","{'x': 12824.744178126775, 'y': -3270.138164527..."
3458764578194135331,shape,parallelogram,"<p>develop this idea, write 100 words</p>",2024-02-07T11:18:44Z,"{'width': 231.0645545179327, 'height': 96.0998...","{'x': 12934.221538661, 'y': -3031.362518688217..."
3458764578223716357,shape,parallelogram,<p><strong>Summarise</strong> these ideas.</p>...,2024-02-07T15:59:57Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': 13792.759167064049, 'y': -3127.283894197..."
3458764578224803182,shape,parallelogram,<p>Does this idea mirror any thoughts of state...,2024-02-07T12:29:22Z,"{'width': 263.84845322298946, 'height': 122.98...","{'x': 12851.355463488451, 'y': -2604.586873605..."


### Find connectors of prompts-shapes (connectors of prallelograms)

In [158]:
def build_output_shape(shape_id, contents = 'Waiting for AI to answer'):

  global shapes_df

  shape = shapes_df.loc [ shape_id ]

  pos = shape['position']
  geo = shape['geometry']

  size = max(geo['width'], geo['height'])

  response = miro_client.create_stiky_note( contents, (pos['x'] +  geo['width'] + size, pos['y'], size, size) )
  a_id = response.json()['id']

  response = miro_client.create_connector( shape_id,  a_id, 'AI answer' )

  #TODO: OPTIMIZE!!
  shapes_df = miro_client.all_items_as_df()

  return a_id

# build_output_shape('3458764578079690837', "TESTCREATE")


def get_output_shape_id(prompt_id):
  out_links = connectors_df[ connectors_df['id_from']==prompt_id]

  if len(out_links)<1:
    a_id = build_output_shape(prompt_id, contents = 'Waiting for AI to answer')
    return a_id
    print("need to create output!!")
  else:
    _items = shapes_df.loc [ out_links['id_to'] ]
    return _items.iloc[0].name


def update_widget_with_ai_answer(prompt_id, answer_text):
  out_links = connectors_df[ connectors_df['id_from']==prompt_id]
  print(len(out_links), out_links)
  if len(out_links)<1:
    print("need to create output!!")
  else:
    _items = shapes_df.loc [ out_links['id_to'] ]
    out_shape_id = _items.iloc[0].name
    # print(out_shape_id)
    # print(out_shape_id, _items.iloc[0])

    res = miro_client.update_item( out_shape_id, item_type=shapes_df.at[out_shape_id, 'type'], content=answer_text,  shape='round_rectangle')
    print(res)


# update_widget_with_ai_answer('3458764578223716357', "test text") #empty
# update_widget_with_ai_answer('3458764578079690837', "test text")

In [160]:
def get_incoming_text(incoming_links):

  incoming_items = shapes_df.loc [ incoming_links['id_from'] ]
  incoming_items = incoming_items.sort_values(by='position', key=lambda x: x.map(lambda d: d['y']), ascending=True)

  concatenated_text = '\n\n'.join(incoming_items.contents)
  # print('ii---- ',len(incoming_items), concatenated_text)

  return concatenated_text

def get_color_for_state(out_shape_id, state):
  item_type = shapes_df.at[out_shape_id, 'type']

  if state==0:
    if item_type=='sticky_note':
      color = 'red'
    else:
      color = '#ff5566'

  if state==1:
    if item_type=='sticky_note':
      color = 'light_green'
    else:
      color = '#ccffdd'

  return color


def build_answer(prompt_id):
  _propmt_text = shapes_df.loc[prompt_id]['contents']

  incoming_links = connectors_df[ connectors_df['id_to']==prompt_id]

  # if there are more than 1 input, merge them
  in_text = get_incoming_text(incoming_links)

  out_shape_id = get_output_shape_id(prompt_id)
  if out_shape_id is not None:

    item_type = shapes_df.at[out_shape_id, 'type']

    color = get_color_for_state(out_shape_id, 0)
    res = miro_client.update_item( out_shape_id, item_type=item_type, content='waiting for AI to answer ....', color = color)
    answer_text = ask_ai(_propmt_text, in_text)
    # display(HTML(r))
    print(r)

    # update_widget_with_ai_answer(prompt_id, answer_text)
    color = get_color_for_state(out_shape_id, 1)
    res = miro_client.update_item( out_shape_id, item_type=item_type, content=answer_text,  shape='round_rectangle', color = color)

  return r


#TODO: sort propmpts first
for prompt_id in prompts_df.index:
  answer_text = build_answer(prompt_id)



{'data': {'content': 'waiting for AI to answer ....'}, 'style': {'fillColor': 'red'}}
{
  "id" : "3458764578250671917",
  "type" : "sticky_note",
  "data" : {
    "content" : "waiting for AI to answer ....",
    "shape" : "rectangle"
  },
  "style" : {
    "fillColor" : "red",
    "textAlign" : "left",
    "textAlignVertical" : "middle"
  },
  "geometry" : {
    "width" : 532.4153298152145,
    "height" : 346.8305577081968
  },
  "position" : {
    "x" : 13347.316856001244,
    "y" : -2605.1515981785724,
    "origin" : "center",
    "relativeTo" : "canvas_center"
  },
  "links" : {
    "self" : "http://api.miro.com/v2/boards/uXjVN61x8Pg%3D/sticky_notes/3458764578250671917"
  },
  "createdAt" : "2024-02-07T15:02:35Z",
  "createdBy" : {
    "id" : "3458764575386404073",
    "type" : "user"
  },
  "modifiedAt" : "2024-02-07T16:18:13Z",
  "modifiedBy" : {
    "id" : "3458764575386404073",
    "type" : "user"
  }
}
{'size': 50, 'limit': 50, 'total': 245, 'data': [{'id': '3458764575387244511